In [1]:
from gurobipy import *

In [2]:
m1 = Model('VRPTW')

Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-11


In [3]:
from operator import le
from unicodedata import name
from numpy import conjugate
import random
from gurobipy import *
m1 = Model('vrptw')

# 20 customer -> 1.1 minute
# 30 customers -> 5 minute    1.69 minutes
# 40                          2.45 min
# 50 customer -> 160 minute


def discreteTime(timeWindow, intervalSize):   # O(lenghtOftimeWindow/size)
    tw = []
    currtime = timeWindow[0]
    while (currtime <= timeWindow[1]):
        tw.append(currtime)
        currtime = currtime+intervalSize
    return tw

customers = 30

# timeWindows = [[20, 30], [30, 40], [30, 40]]

timeWindows = [[170, 200], [80, 210], [410, 490], [80, 190], [290, 350], [280, 390], [160, 260], [460, 530], [450, 640], [180, 190], [500, 550], [140, 340], [160, 290], [180, 240], [160, 310], [490, 580], [460, 560], [190, 350], [310, 500], [190, 320], [10, 140], [140, 320], [140, 250], [270, 470], [40, 170], [330, 520], [430, 490], [480, 610], [410, 600], [190, 300]]
timeMatrix = [[0, 30, 100, 60, 50, 20, 80, 70, 90, 60, 100, 40, 100, 50, 50, 50, 30, 10, 70, 80, 30, 30, 70, 60, 10, 60, 50, 20, 40, 20], [30, 0, 30, 40, 30, 40, 60, 90, 100, 10, 40, 50, 80, 70, 40, 20, 30, 70, 50, 40, 30, 70, 80, 20, 30, 90, 80, 30, 80, 60], [100, 30, 0, 30, 80, 100, 60, 10, 90, 20, 50, 60, 40, 100, 50, 70, 60, 60, 20, 70, 100, 80, 40, 50, 20, 80, 70, 50, 20, 70], [60, 40, 30, 0, 20, 30, 80, 20, 70, 40, 70, 60, 40, 70, 20, 100, 50, 70, 70, 80, 20, 70, 20, 10, 60, 60, 60, 60, 10, 90], [50, 30, 80, 20, 0, 20, 20, 20, 100, 60, 10, 40, 70, 100, 10, 50, 70, 70, 90, 80, 80, 60, 20, 100, 80, 20, 50, 80, 10, 80], [20, 40, 100, 30, 20, 0, 80, 20, 30, 30, 80, 10, 50, 10, 100, 20, 20, 20, 100, 80, 30, 30, 90, 40, 80, 10, 40, 30, 60, 40], [80, 60, 60, 80, 20, 80, 0, 60, 10, 10, 80, 50, 70, 80, 20, 50, 40, 20, 90, 90, 30, 70, 50, 50, 10, 20, 20, 60, 80, 30], [70, 90, 10, 20, 20, 20, 60, 0, 30, 20, 90, 70, 70, 20, 30, 50, 40, 80, 30, 50, 100, 50, 40, 90, 30, 70, 10, 50, 50, 50], [90, 100, 90, 70, 100, 30, 10, 30, 0, 100, 20, 40, 10, 30, 70, 80, 50, 80, 30, 20, 100, 80, 100, 30, 80, 30, 100, 10, 50, 40], [60, 10, 20, 40, 60, 30, 10, 20, 100, 0, 70, 20, 30, 70, 90, 20, 100, 70, 70, 80, 50, 90, 30, 40, 40, 40, 40, 60, 100, 10], [100, 40, 50, 70, 10, 80, 80, 90, 20, 70, 0, 20, 100, 100, 30, 30, 50, 60, 90, 20, 40, 100, 10, 50, 90, 30, 80, 70, 100, 30], [40, 50, 60, 60, 40, 10, 50, 70, 40, 20, 20, 0, 20, 60, 100, 60, 70, 80, 90, 50, 100, 40, 30, 60, 60, 90, 40, 30, 50, 60], [100, 80, 40, 40, 70, 50, 70, 70, 10, 30, 100, 20, 0, 40, 40, 80, 50, 90, 90, 100, 60, 50, 40, 20, 10, 60, 40, 20, 40, 50], [50, 70, 100, 70, 100, 10, 80, 20, 30, 70, 100, 60, 40, 0, 100, 30, 70, 20, 80, 70, 100, 60, 40, 10, 80, 20, 100, 10, 100, 100], [50, 40, 50, 20, 10, 100, 20, 30, 70, 90, 30, 100, 40, 100, 0, 100, 30, 20, 60, 50, 80, 90, 20, 60, 90, 80, 90, 90, 80, 10], [50, 20, 70, 100, 
50, 20, 50, 50, 80, 20, 30, 60, 80, 30, 100, 0, 40, 30, 40, 10, 80, 80, 10, 50, 50, 50, 90, 70, 30, 30], [30, 30, 60, 50, 70, 20, 40, 40, 50, 100, 50, 70, 50, 70, 30, 40, 0, 70, 50, 10, 10, 80, 50, 60, 100, 30, 30, 40, 100, 10], [10, 70, 60, 70, 70, 20, 20, 80, 80, 70, 60, 80, 90, 20, 20, 30, 70, 0, 40, 30, 80, 60, 100, 30, 40, 30, 90, 40, 30, 20], [70, 50, 20, 70, 90, 100, 90, 30, 30, 70, 90, 90, 90, 80, 60, 40, 50, 40, 0, 50, 40, 80, 50, 90, 30, 100, 100, 30, 100, 60], [80, 40, 70, 80, 80, 80, 90, 50, 20, 80, 20, 50, 100, 70, 50, 10, 10, 30, 50, 0, 60, 70, 20, 40, 80, 40, 90, 80, 90, 40], [30, 30, 100, 20, 80, 30, 30, 100, 100, 50, 40, 100, 60, 100, 80, 80, 10, 80, 40, 60, 0, 40, 30, 30, 20, 100, 100, 60, 80, 10], [30, 70, 80, 70, 60, 30, 70, 50, 80, 90, 100, 40, 50, 60, 90, 80, 80, 60, 80, 70, 40, 0, 10, 100, 20, 50, 70, 50, 30, 100], [70, 80, 40, 20, 20, 90, 50, 40, 100, 30, 10, 30, 40, 40, 20, 10, 50, 100, 50, 20, 30, 10, 0, 90, 50, 20, 40, 30, 100, 10], [60, 20, 50, 10, 100, 40, 50, 90, 30, 40, 50, 60, 20, 10, 
60, 50, 60, 30, 90, 40, 30, 100, 90, 0, 40, 80, 90, 50, 60, 50], [10, 30, 20, 60, 80, 80, 10, 30, 80, 40, 90, 60, 10, 80, 90, 50, 100, 40, 30, 80, 20, 20, 50, 40, 0, 70, 50, 10, 70, 50], [60, 90, 80, 60, 20, 10, 20, 70, 30, 40, 30, 90, 60, 20, 80, 50, 30, 30, 100, 40, 100, 50, 20, 80, 70, 0, 40, 40, 40, 20], [50, 80, 70, 60, 50, 40, 20, 10, 100, 40, 80, 40, 40, 100, 90, 90, 30, 90, 100, 90, 100, 70, 40, 90, 50, 40, 0, 80, 30, 10], [20, 30, 50, 60, 80, 30, 60, 50, 10, 60, 70, 30, 20, 10, 90, 70, 40, 40, 30, 80, 
60, 50, 30, 50, 10, 40, 80, 0, 50, 50], [40, 80, 20, 10, 10, 60, 80, 50, 50, 100, 100, 50, 40, 100, 80, 30, 100, 30, 100, 90, 80, 30, 100, 60, 70, 40, 30, 50, 0, 40], [20, 
60, 70, 90, 80, 40, 30, 50, 40, 10, 30, 60, 50, 100, 10, 30, 10, 20, 60, 40, 10, 100, 10, 50, 50, 20, 10, 50, 40, 0]]

# timeWindows = []
# for i in range(0,customers):
#     st = random.randint(1,2)*10
#     et = random.randint(1,5)*10 + st
#     tw = [st,et]
#     timeWindows.append(tw)

# print("time Windows ", timeWindows)
# print()

discreteTimeWindows = []
for i in timeWindows:
    discreteTimeWindows.append(discreteTime(i, 10))

print("discreteTimeWindows", discreteTimeWindows)
print()

# starting_time = '6:00am'

# timeMatrix = [[0, 30, 10], [30, 0, 10], [10, 10, 0]]

# timeMatrix = []
# for i in range(0,customers):
#     timeMatrix.append([0]*customers)
# for i in range(0,customers):
#     for j in range(0,customers):
#         if(i==j):
#             timeMatrix[i][j] = 0
#         elif (i<j):
#             timeMatrix[i][j] = random.randint(1,10)*10
#             timeMatrix[j][i] = timeMatrix[i][j]

# print("timeMatrix", timeMatrix)
# print()


discreteTimeWindows [[170, 180, 190, 200], [80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210], [410, 420, 430, 440, 450, 460, 470, 480, 490], [80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190], [290, 300, 310, 320, 330, 340, 350], [280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390], [160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260], [460, 470, 480, 490, 500, 510, 520, 530], [450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640], [180, 190], [500, 510, 520, 530, 540, 550], [140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340], [160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290], [180, 190, 200, 210, 220, 230, 240], [160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310], [490, 500, 510, 520, 530, 540, 550, 560, 570, 580], [460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560], [190, 200, 210, 220, 230, 240

In [4]:


count = 0
nodeCount = []  # assigns a number to node using count and stores its value

for i in range(0,len(discreteTimeWindows)):
    c = []
    for j in range(0, len(discreteTimeWindows[i])):
        c.append(count+1)
        count += 1
    nodeCount.append(c)

print("Node Count ", nodeCount)
print()


edgeFlows = {}
edgeNumber = 0
x = [] 
edges = {}
edgesList = []  # stores edge number going out of ith node
edgesList.append([])  # for s
# solver = pywraplp.Solver.CreateSolver('GLOP')

Node Count  [[1, 2, 3, 4], [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18], [19, 20, 21, 22, 23, 24, 25, 26, 27], [28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], [40, 41, 42, 43, 44, 45, 46], [47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58], [59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69], [70, 71, 72, 73, 74, 75, 76, 77], [78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97], [98, 99], [100, 101, 102, 103, 104, 105], [106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126], [127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140], [141, 142, 143, 144, 145, 146, 147], [148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163], [164, 165, 166, 167, 168, 169, 170, 171, 172, 173], [174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184], [185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201], [202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212

In [5]:
# Graph Construction

# out of s -> assuming every node is reachable from s
e =[]
for i in range(0,len(nodeCount)):
    for j in range(0,len(nodeCount[i])):
        e.append(edgeNumber)
        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber ))
        edges['x[%i]' % edgeNumber] = (0,nodeCount[i][j])
        edgeNumber+=1
edgesList[0]=e

for i in range(0,len(discreteTimeWindows)):
        for j in range(0,len(discreteTimeWindows[i])):    # j th node
            u = nodeCount[i][j]
            utime = discreteTimeWindows[i][j]
            e = []  # stores edges reachable from u
            for k in range(0,len(discreteTimeWindows)):
                for l in range(len(discreteTimeWindows[k])-1,-1,-1): # l th node
                    v = nodeCount[k][l]
                    vtime = discreteTimeWindows[k][l]
                    if(i!=k and utime+timeMatrix[i][k]<=vtime):
                        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber ))
                        edges['x[%i]' % edgeNumber] = (u,v) # u -> v reachable
                        e.append(edgeNumber)
                        edgeNumber += 1
                    elif (utime+timeMatrix[i][k]>vtime):
                        break
            edgesList.append(e)

# into t
e=[]
for i in range(0,len(nodeCount)):
    for j in range(0,len(nodeCount[i])):
            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber ))
            edges['x[%i]' % edgeNumber] = (nodeCount[i][j],-1)
            #add edge number in edge list 
            edgesList[nodeCount[i][j]].append(edgeNumber)
            edgeNumber+=1
edgesList.append([])
print("Node Count after s and t:",nodeCount)
print()


m1.update()

print('edge list', edgesList)
print()
print("edges", edges)
print()
print('edge number', edgeNumber)
print()

Node Count after s and t: [[1, 2, 3, 4], [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18], [19, 20, 21, 22, 23, 24, 25, 26, 27], [28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], [40, 41, 42, 43, 44, 45, 46], [47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58], [59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69], [70, 71, 72, 73, 74, 75, 76, 77], [78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97], [98, 99], [100, 101, 102, 103, 104, 105], [106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126], [127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140], [141, 142, 143, 144, 145, 146, 147], [148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163], [164, 165, 166, 167, 168, 169, 170, 171, 172, 173], [174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184], [185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201], [202, 203, 204, 205, 206, 207, 208, 209,

In [6]:
# constraints
noc = 0
# contraint for outgoing flow for a customer
   
for i in range(0,len(nodeCount)): # choose one list from nodeCount
    coeff = [] # store where coeff = 1
    for j in range(0,len(nodeCount[i])): # se
        node = nodeCount[i][j]
        for k in range(0,len(edgesList[node])):
            e = edgesList[node][k]
            coeff.append(x[e])
    m1.addConstr(( sum(coeff) == 1  ))
    noc+=1
        

# constraint for incoming for a customer
incomingEdgeList =[]
for i in range(0,len(edgesList)):
        incomingEdgeList.append([]) # adding lists in a list
for i in edges:
        v = edges[i][1]  # u -> v
        edge_no = int(i[2:-1])
        incomingEdgeList[v].append(edge_no)

print("incoming edge list: ",incomingEdgeList)
print()

for i in range(0,len(nodeCount)):
    contraint = [0] * (edgeNumber) # indicates number of edges
    coeff = []
    for j in range(0,len(nodeCount[i])):
        node = nodeCount[i][j]
        for k in range(0,len(incomingEdgeList[node])):
            e = incomingEdgeList[node][k]
            contraint[e] = 1
            coeff.append(x[e])

    m1.addConstr(( sum(coeff) == 1  ))
    noc+=1

# constraint for a node incoming = outgoing
for i in range(0,len(nodeCount)):
    for j in range(0,len(nodeCount[i])):
        contraint = [0] * (edgeNumber)
        coeff = []
        node = nodeCount[i][j]
        for k in range(0,len(incomingEdgeList[node])):
            e = incomingEdgeList[node][k]
            contraint[e] = 1
            coeff.append(x[e])
        for k in range(0,len(edgesList[node])):
            e = edgesList[node][k]
            contraint[e] = -1
            coeff.append(-x[e])
        m1.addConstr(( sum(coeff) == 0  ))
        noc+=1

incoming edge list:  [[], [0, 1415, 1765, 2109, 2446, 2775, 3095, 3405, 6091, 6443, 6789, 7129, 36349, 36723, 37097, 37470, 37842, 38212, 38579, 38942, 39301, 39654, 40001, 40340, 40669, 40988, 41294, 50510, 50877, 51242, 51604, 51963, 52318, 52668, 53012, 53349, 53678, 53998, 54310, 54613], [1, 1414, 1764, 2108, 2445, 2774, 3094, 3404, 3703, 6090, 6442, 6788, 7128, 7461, 16246, 36348, 36722, 37096, 37469, 37841, 38211, 38578, 38941, 39300, 39653, 40000, 40339, 40668, 40987, 41293, 41579, 50509, 50876, 51241, 51603, 51962, 52317, 52667, 53011, 53348, 53677, 53997, 54309, 54612, 54903], [2, 1413, 1763, 2107, 2444, 2773, 3093, 3403, 3702, 3989, 6089, 6441, 6787, 7127, 7460, 7783, 16245, 16536, 36347, 36721, 37095, 37468, 37840, 38210, 38577, 38940, 39299, 39652, 39999, 40338, 40667, 40986, 41292, 41578, 41852, 50508, 50875, 51240, 51602, 51961, 52316, 52666, 53010, 53347, 53676, 53996, 54308, 54611, 54902], [3, 1412, 1762, 2106, 2443, 2772, 3092, 3402, 3701, 3988, 4263, 6088, 6440, 6786,

In [7]:
objectivefun = []
for i in range(0, len(edgesList[0])):
    objectivefun.append(x[edgesList[0][i]])
obj = sum(objectivefun)
m1.setObjective(obj,GRB.MINIMIZE)
                
    
m1.optimize()
print("Runtime is :",m1.Runtime)
print("objective value : ",m1.objVal)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 448 rows, 60793 columns and 241620 nonzeros
Model fingerprint: 0x04cf8730
Variable types: 0 continuous, 60793 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 12.0000000
Presolve time: 0.51s
Presolved: 448 rows, 60793 columns, 237622 nonzeros
Variable types: 0 continuous, 60793 integer (60793 binary)

Root relaxation: objective 1.500000e+00, 2515 iterations, 1.73 seconds (1.10 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.50000    0   75   12.00000    1.50000  87.5%     -    4s
H    0     0                       4.0000000    1.50000  62.5%

In [8]:
for v in m1.getVars():
    print('%s: %g' % (v.varName,v.x))

x[0]: -0
x[1]: -0
x[2]: -0
x[3]: -0
x[4]: -0
x[5]: -0
x[6]: -0
x[7]: 1
x[8]: -0
x[9]: -0
x[10]: -0
x[11]: -0
x[12]: -0
x[13]: -0
x[14]: -0
x[15]: -0
x[16]: -0
x[17]: -0
x[18]: -0
x[19]: -0
x[20]: -0
x[21]: -0
x[22]: -0
x[23]: -0
x[24]: -0
x[25]: -0
x[26]: -0
x[27]: -0
x[28]: -0
x[29]: -0
x[30]: -0
x[31]: -0
x[32]: -0
x[33]: -0
x[34]: -0
x[35]: -0
x[36]: -0
x[37]: -0
x[38]: -0
x[39]: -0
x[40]: -0
x[41]: -0
x[42]: -0
x[43]: -0
x[44]: -0
x[45]: -0
x[46]: -0
x[47]: -0
x[48]: -0
x[49]: -0
x[50]: -0
x[51]: -0
x[52]: -0
x[53]: -0
x[54]: -0
x[55]: -0
x[56]: -0
x[57]: -0
x[58]: -0
x[59]: 0
x[60]: -0
x[61]: -0
x[62]: -0
x[63]: -0
x[64]: -0
x[65]: -0
x[66]: -0
x[67]: -0
x[68]: -0
x[69]: -0
x[70]: -0
x[71]: -0
x[72]: -0
x[73]: -0
x[74]: -0
x[75]: -0
x[76]: -0
x[77]: -0
x[78]: -0
x[79]: -0
x[80]: -0
x[81]: -0
x[82]: -0
x[83]: -0
x[84]: -0
x[85]: -0
x[86]: -0
x[87]: -0
x[88]: -0
x[89]: -0
x[90]: -0
x[91]: -0
x[92]: -0
x[93]: -0
x[94]: -0
x[95]: -0
x[96]: -0
x[97]: -0
x[98]: -0
x[99]: -0
x[100]: -0
x

In [9]:
print("Number of Constraints :",noc)

Number of Constraints : 448
